# Kernel Regression

## Loading Packages

In [1]:
include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")
include("jlFiles/OlsFn.jl")

OlsFn

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Loading Data

The data consists of daily returns on U.S. large caps. 

We then regress the daily return on the lagged daily return, that is, a non-parametric AR(1)

$
R_t = b(R_{t-1}) + \epsilon_t
$

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',skipstart=1)
R   = xx[:,11]            #returns for the size portfolio we want to study
xx  = nothing

y   = R[2:end]            #dependent variable
x   = R[1:end-1]          #regressor
T   = size(x,1)

println("Sample size: $T")

Sample size: 8324



A kernel regression is of the form
$$
\hat{b}(x)=\frac{\sum\nolimits_{t=1}^{T}w_{t}(x)y_{t}}{\sum\nolimits_{t=1}^{T}w_{t}(x)},
$$
where $w_{t}(x)$ is the weight of observation $t$, defined by a kernel function.

The function KernRegNormalFn uses a normal kernel, $N(x,h^{2})$.

In [4]:
function KernRegNormalFn(y,x,xGrid,h,vv)

  Ngrid = length(xGrid)                  #number of grid points

  bHat = fill(NaN,Ngrid)                 #b[x(t)]
  for i = 1:Ngrid                        #loop over elements in xGrid
    zi      = (x .- xGrid[i])/h
    w       = exp.(-zi.^2/2)./(h*sqrt(2*pi))       #gaussian kernel, with "std" = h
    bHat[i] = sum(w[vv].*y[vv])/sum(w[vv])         #sum over observations (data)
  end

  return bHat

end

KernRegNormalFn (generic function with 1 method)

In [5]:
xGrid = collect(-10:0.25:10)
h     = 1.5
bHat    = KernRegNormalFn(y,x,xGrid,h,1:T)         #baseline choice of h
bHatHih = KernRegNormalFn(y,x,xGrid,h*2,1:T)       #high h
bHatLoh = KernRegNormalFn(y,x,xGrid,h*0.5,1:T)     #low h


ha = plot(xGrid,bHatLoh,line=(:dot,2),label="low h")
plot!(xGrid,bHat,label="medium h")
plot!(xGrid,bHatHih,line=(:dash,2),label="high h")
plot!(xlim=(minimum(xGrid),maximum(xGrid)),ylim=(-5,5))
title!("Return vs lagged return, kernel regression")
xlabel!("Lagged return")
ylabel!("Return")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 Return vs lagged return, kernel regression 
 
 
 Lagged return 
 
 
 Return 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 54.8916,49.6484 61.4594,51.012 68.0272,52.7428 74.595,54.9986 81.1628,58.0195 87.7306,62.1431 94.2984,67.7637 100.866,75.1408 107.434,83.9903 114.002,93.1389 
 120.57,101.06 127.137,107.38 133.705,113.592 140.273,121.997 146.841,134.196 153.408,150.105 159.976,167.543 166.544,182.789 173.112,192.588 179.68,196.268 
 186.247,195.774 192.815,193.826 199.383,192.295 205.951,191.79 212.519,192.07 219.086,192.665 225.654,193.254 232.222,193.717 238.79,194.039 245.358,194.24 
 251.925,194.351 258.493,194.406 265.061,194.436 271.629,194.462 278.197,194.49 284.764,194.516 291.332,194.538 297.9,194.553 304.468,194.558 311.035,194.553 
 317.603,194.534 324.171,194.5 330.739,194.45 337.307,194.388 343.874,194.316 350.442,194.238 357.01,194.149 363.578,194.028 370.146,193.834 376.713,193.513 
 383.281,193.017 389.849,192.337 396.417,191.564 402.985,190.942 409.552,190.894 416.12,191.93 422.688,194.403 429.256,198.256 435.824,202.947 442.391,207.582 
 448.959,211.092 455.527,212.397 462.095,210.622 468.663,205.395 475.23,197.055 481.798,186.53 488.366,174.991 494.934,163.689 501.501,154.269 508.069,149.514 
 514.637,153.934 521.205,172.31 527.773,204.337 534.34,240.804 540.908,270.192 547.476,286.923 554.044,290.348 560.612,281.695 567.179,265.705 573.747,250.388 
 580.315,240.304 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.8916,74.4604 61.4594,77.9403 68.0272,81.9429 74.595,86.5436 81.1628,91.8167 87.7306,97.827 94.2984,104.617 100.866,112.191 107.434,120.492 114.002,129.387 
 120.57,138.649 127.137,147.966 133.705,156.964 140.273,165.262 146.841,172.535 153.408,178.58 159.976,183.339 166.544,186.898 173.112,189.435 179.68,191.171 
 186.247,192.321 192.815,193.066 199.383,193.544 205.951,193.851 212.519,194.051 219.086,194.185 225.654,194.277 232.222,194.342 238.79,194.389 245.358,194.423 
 251.925,194.448 258.493,194.466 265.061,194.478 271.629,194.486 278.197,194.49 284.764,194.49 291.332,194.488 297.9,194.483 304.468,194.474 311.035,194.464 
 317.603,194.45 324.171,194.433 330.739,194.412 337.307,194.387 343.874,194.359 350.442,194.325 357.01,194.286 363.578,194.242 370.146,194.191 376.713,194.135 
 383.281,194.075 389.849,194.013 396.417,193.953 402.985,193.902 409.552,193.87 416.12,193.869 422.688,193.916 429.256,194.03 435.824,194.226 442.391,194.514 
 448.959,194.892 455.527,195.332 462.095,195.782 468.663,196.162 475.23,196.376 481.798,196.332 488.366,195.973 494.934,195.305 501.501,194.418 508.069,193.496 
 514.637,192.804 521.205,192.669 527.773,193.434 534.34,195.401 540.908,198.755 547.476,203.468 554.044,209.237 560.612,215.478 567.179,221.443 573.747,226.43 
 580.315,229.998 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 54.8916,182.341 61.4594,184.107 68.0272,185.646 74.595,186.98 81.1628,188.129 87.7306,189.115 94.2984,189.957 100.866,190.674 107.434,191.282 114.002,191.796 
 120.57,192.23 127.137,192.595 133.705,192.902 140.273,193.159 146.841,193.374 153.408,193.554 159.976,193.704 166.544,193.829 173.112,193.933 179.68,194.02 
 186.247,194.091 192.815,194.151 199.383,194.2 205.951,194.241 212.519,194.274 219.086,194.301 225.654,194.324 232.222,194.342 238.79,194.356 245.358,194.367 
 251.925,194.376 258.493,194.383 265.061,194.387 271.629,194.39 278.197,194.392 284.764,194.393 291.332,194.392 297.9,194.391 304.468,194.388 311.035,194.385 
 317.603,194.382 324.171,194.378 330.739,194.373 337.307,194.368 343.874,194.363 350.4

## Rule of Thumb Choice of h

Run the regression 

$
y=\alpha+\beta x+\gamma x^{2}+\varepsilon
$

and use the following rule-of-thumb choice

$
h  =T^{-1/5}|\gamma|^{-2/5}\sigma_{\varepsilon}^{2/5}(x_{\max}-x_{\min})^{1/5}\times0.6.
$

In practice, replace $x_{\max}-x_{\min}$ by the difference between the 90th
and 10th percentiles of $x$.

In [6]:
(b,res,) = OlsFn(y,[x.^2 x ones(T)])             #rule of thumb bandwidth

sigma = std(res)
gamm  = b[1]
xSort = sort(x)
x_10  = xSort[floor(Int,T*0.1)]          #crude 10th and 90th percentiles
x_90  = xSort[floor(Int,T*0.9)]

h_crude = 0.6*sigma^(2/5)*abs(gamm)^(-2/5)*(x_90-x_10)^(1/5)*T^(-1/5)

printlnPs("\nRule-of-thumb value of h: ",h_crude)


Rule-of-thumb value of h:      0.667


## Cross-Validation (leave-one-out)

To do a cross-validation 

(1) Pick a bandwidth $h$, do the kernel regression
but leave out observation $t$ and then record the out-of-sample prediction error $y_{t}-\hat{b}_{-t}
(x_{t},h)$. Notice that this is the error for observation $t$ only. 

(2) Repeat for all $t=1-T$ to calculate the EPE

$
\text{EPE}(h)=\sum\nolimits_{t=1}^{T}\left[  y_{t}-\hat{b}_{-t}(x_{t},h)\right]  ^{2}/T,
$

(3) Finally, redo for several different values of $h$---and pick the one
that minimizes EPE$(h)$. 

In [7]:
println("\nCross-validation calculations take some time")

hM = h_crude*[0.5;0.75;1;1.5;2;3;4;5;10]

Nh   = length(hM)
EPEM = fill(NaN,(T,Nh))
for t = 1:T
    v_No_t = setdiff(1:T,t)                  #exclude t from estimation
    for j = 1:Nh
        b_t       = KernRegNormalFn(y,x,x[t],hM[j],v_No_t)
        EPEM[t,j] = (y[t] - b_t[1])^2
    end
end
EPE = mean(EPEM,1)'
println("h and EPE:")
printmat([hM EPE])


Cross-validation calculations take some time
h and EPE:
     0.334     1.314
     0.501     1.302
     0.667     1.295
     1.001     1.288
     1.335     1.286
     2.002     1.287
     2.670     1.291
     3.337     1.292
     6.674     1.296



In [8]:
ha = plot(hM,EPE/minimum(EPE),legend=false)
plot!(xlim=(minimum(hM)-0.05,maximum(hM)+0.05),ylim= (0.99,1.025))
title!("Cross validation simulations, kernel regression")
xlabel!("Bandwidth")
ylabel!("EPE/minimum(EPE)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 0.99 
 
 
 1.00 
 
 
 1.01 
 
 
 1.02 
 
 
 Cross validation simulations, kernel regression 
 
 
 Bandwidth 
 
 
 EPE/minimum(EPE)